In [1]:
%pip install PyMuPDF
%pip install PIL
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pytesseract

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install python-docx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [87]:
import os
import pytesseract
from PIL import Image
import numpy
import docx
import fitz
import logging
import cv2

logging.basicConfig(filename="Task1.log", level=logging.INFO,  format='%(asctime)s %(message)s')

In [88]:
pytesseract.pytesseract.tesseract_cmd = r'C:\\Users\\tr4\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe'

In [89]:
def file_type(file_path):
    _, file_extension = os.path.splitext(file_path)
    return file_extension.lower()

In [90]:

def extract_from_image(file_path):
    # image = Image.open(file_path)
    import numpy as np
    image = cv2.imread(file_path)
    logging.info("Extracting the Text from Images")
    # norm_img = np.zeros((image.shape[0], image.shape[1]))
    # image = cv2.normalize(image, norm_img, 0, 255, cv2.NORM_MINMAX)

    # length_x, width_y = image.size
    # factor = min(1, float(1024.0 / length_x))
    # size = int(factor * length_x), int(factor * width_y)
    # im_resized = image.resize(size, Image.ANTIALIAS)
    # im_resized.save(image, dpi=(300, 300))

    # image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15)


    # kernel = np.ones((5,5),np.uint8)
    # image = cv2.erode(image, kernel, iterations = 1)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # kernel = numpy.ones((2,2),numpy.uint8)
    # image = cv2.erode(image, kernel, iterations=1)
    # image = cv2.bitwise_not(image)
    # image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 11)
    text = pytesseract.image_to_string(image)
    # text = ''.join(text.split())
    return text 

In [91]:
import io

In [92]:
def extract_from_pdf(file_path):
    logging.info("Extracting the text from the PDF")
    doc = fitz.open(file_path)
    images = []
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
        image_list = page.get_images(full=True)
        
        for img_index, img in enumerate(image_list):
            logging.info("Images found in the PDF")
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image = Image.open(io.BytesIO(image_bytes))
            images.append(image)
    
    logging.info("The total number of Images found in the PDF", len(images))
    for img in images:
        logging.info("Text Extracted from the Images")
        ocr_text = pytesseract.image_to_string(img)
        text += "\n" + ocr_text
    return text

In [93]:
def extract_from_docx(file_path):
    logging.info("Extracting the text from the Document")
    doc = docx.Document(file_path)
    text = " ".join([para.text for para in doc.paragraphs])
    images = []
    for rel in doc.part.rels:
        if "image" in doc.part.rels[rel].target_ref:
            image = doc.part.rels[rel].target_part.blob
            img = Image.open(io.BytesIO(image))
            images.append(img)
    logging.info(f"The total number of Images found in the document {len(images)}")
    for img in images:
        logging.info("Text Extracted from the Images")
        ocr_text = pytesseract.image_to_string(img)
        text += "\n" + ocr_text
    return text

In [94]:
def extract_from_txt(file_path):
    logging.info("Extracting the text from the Text file")
    with open(file_path,'r') as file:
        text = file.read()
    return text

In [101]:
def valid_characters(text):
    try:
        text.encode('utf-8')
        return True
    except UnicodeEncodeError:
        return False

In [102]:
def process_file(file_path):
    file_name = os.path.basename(file_path)
    file_new = file_type(file_path)
    if file_new in ['.jpg', '.jpeg', '.png']:
        logging.info(f"Found an Image file: {file_name}")
        extracted_text = extract_from_image(file_path)
    elif file_new == '.pdf':
        logging.info(f"Found a PDF file: {file_name}")
        extracted_text = extract_from_pdf(file_path)
    elif file_new in ['.docx', '.doc']:
        logging.info(f"Found a document file: {file_name}")
        extracted_text = extract_from_docx(file_path)
    elif file_new == '.txt':
        logging.info(f"Found a Text file: {file_name}")
        extracted_text = extract_from_txt(file_path)
    else:
        logging.error(f"Unsupported file type: {file_new} - {file_name}")
        extracted_text = None
    
    if extracted_text:
        if not valid_characters(extracted_text):
            logging.warning(f"Extracted text contains invalid characters: {file_name}")

    return file_name, extracted_text


In [98]:
def process_folder(input_folder):
    output_folder = "Output"
    os.makedirs(output_folder, exist_ok=True)
    files = os.listdir(input_folder)
    for file_name in files:
        file_path = os.path.join(input_folder, file_name)
        if os.path.isfile(file_path):
            original_name, extracted_text = process_file(file_path)
            if extracted_text:
                output_file_name = f"{os.path.splitext(original_name)[0]}_extracted.txt"
                output_file_path = os.path.join(output_folder, output_file_name)
                with open(output_file_path, 'w', encoding='utf-8') as f:
                    f.write(extracted_text)
                print(f"Extracted text saved to: {output_file_path}")
            else:
                print(f"Extraction failed for: {file_path}")

if __name__ == "__main__":
    input_folder = "Input"
    process_folder(input_folder)

Extracted text saved to: Output\Aashirvad_extracted.txt
Extracted text saved to: Output\image1_extracted.txt
Extracted text saved to: Output\image2_extracted.txt
Extracted text saved to: Output\image3_extracted.txt
Extracted text saved to: Output\ImagesWord_extracted.txt
Extracted text saved to: Output\Task_interns_extracted.txt
Extracted text saved to: Output\withImagesWord_extracted.txt
